<a href="https://colab.research.google.com/github/rmontteiro/ML_IGTI_bootcamp/blob/master/handsonMod4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

APLICATIVO PARA VERIFICAR SE O SER VIVO É QUADRUPEDE OU BIPEDE



In [ ]:
# Quadrupede = 1
# Bipede = -1 
# cao = [-1,-1,1,1] resposta = 1
# gato = [1,1,1,1] resposta = 1
# cavalo = [1,1,-1,1] resposta = 1
# homem = [-1,-1,-1, 1] resposta = 1


In [1]:
# pesos (sinapses)
w = [0,0,0,0]

# entradas
x = [[-1,-1,1,1],
     [1,1,1,1],
     [1,1,-1,1],
     [-1,-1,-1, 1]]

#respostas esperadas
t = [1,1,1,-1]

#bias (ajuste fino)
b = 0

#saida
y = 0

#numero maximo de interações
max_int = 10

# taxa de aprendizado

taxa_aprendizado = 1

#soma
soma = 0 

#theshold
threshold = 1

#nome do animal
animal = ""

# resposta = acerto ou falha
resposta = ""

In [2]:
# dicionario de dados
d = {'-1,-1,1,1' :'cao',
     '1,1,1,1' : 'gato',
     '1,1,-1,1' : 'cavalo',
     '-1,-1,-1,1' : 'homem'} 

print ('Em Treinamento')

Em Treinamento


In [15]:
# função para converter listas em strings
def listTostring(List):
  s = str(List).strip('[]')
  s = s.replace(' ', '')
  #print ('função' + s)
  return s
# inicio do algoritimo
for k in range(1, max_int):
  acertos = 0
  print('INTERAÇÃO ' + str(k)+'------------------------------')
  for i in range (0, len(x)):
    soma = 0

    #pega o nome do animal no dicionário
    if (listTostring(x[i])) in d:
      animal = d[listTostring(x[i])]
    else:
      animal = ""
    # para calcular saida do perceptron, cada entrada x  é multiplicada pelo seu peso correspondente
    for j in range(0, len(x[i])):
      soma += x[i][j] *w[j]

    #a saida é igual a adição do bias com a soma anterior
    y_in = b + soma 
    # print('y_in = ", str(y_in))

    #função de saida determinada pelo threshold
    if y_in > threshold:
      y = 1
    elif y_in >= -threshold and y_in <= threshold:
      y = 0
    else:
      y = -1
    
    #atualiza os pesos caso a saida  não corresponda  ao valor esperado
    if y == t[i]:
      acertos +=1
      resposta = ' acerto'
    else:
      for j in range (0, len(w)):
        #peso anterior  + (taxa de aprendizado*valor esperado*valor de x no vetor)
        w[j] = w[j] + (taxa_aprendizado * t[i] * x[i][j])
        print ('Novo Peso' + str(j)+' '+str(w[j]))
      #bias ou erro = classe anterior - classe atual
      b = t[i] - y
      resposta = 'Falha - Peso atualizado' + 'Bias = '+ str(b)
    #imprime a resposta
    if y == 1:
      print(animal+'= quadrupede =' + resposta)
    elif y==0:
      print(animal+' = padrão não identificado = ' + resposta)
    elif y==-1:
      print(animal+' = Bípede = ' + resposta)
  
  if acertos ==len(x):
    print('Funcionalidade aprendida com '+str(k)+' interações' )
    break;
  print(' ')
print('Finalizado')


  

    

INTERAÇÃO 1------------------------------
cao= quadrupede =acerto
gato= quadrupede =acerto
cavalo= quadrupede =acerto
homem = Bípede = acerto
Funcionalidade aprendida com 1 interações
Finalizado
